In [1]:
pip install Keras-Applications

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.7 MB/s eta 0:00:00


In [ ]:
import os
from keras.preprocessing.image import  ImageDataGenerator
from keras.layers import Dense,Flatten,GlobalAveragePooling2D,Input,Conv2D,MaxPooling2D,Dropout
from keras.models import Model,load_model,Sequential
from keras.applications.xception import  Xception
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.efficientnet import EfficientNetB7
from keras.applications.efficientnet_v2 import EfficientNetV2L
import keras.callbacks as kcallbacks
import keras
#from keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.utils import img_to_array
import math
import random
from keras.utils import plot_model
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

In [ ]:
#generate training and test images
TARGET_SIZE=(224,224)
INPUT_SIZE=(224,224,3)
BATCHSIZE=32	#could try 128 or 32

#Normalization
train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/Transfer_IDS_IIoT/Datasets/train_A',
        target_size=TARGET_SIZE,
        batch_size=BATCHSIZE,
        class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        '/content/drive/MyDrive/Transfer_IDS_IIoT/Datasets/test_A',
        target_size=TARGET_SIZE,
        batch_size=BATCHSIZE,
        class_mode='categorical')

In [ ]:
#Tuned Hyperparameters for Generic CNN model by Genetic Algorithm
# f1,f2,f3 : Filter Sizes : 32, 128, 256
# k : Kernel_size : 5
# a1, a2: Activation functions : relu, selu
# d1: Drop out : 0.1
# op: Optimizer : adam
# ep: Number of epochs : 17

input_shape=INPUT_SIZE
num_class=15
epochs=25                #Training is executed for 25 epochs for the better visualization of results
savepath='/content/drive/MyDrive/Transfer_IDS_IIoT/Optimized_Trained_Models/generic_cnn.h5'

#Define CNN Model
model = Sequential()
model.add(Conv2D(32,(5,5),strides=(1,1),input_shape=input_shape,padding='same',activation='relu'))
model.add(Conv2D(32,(5,5),strides=(1,1),padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128,(5,5),strides=(1,1),padding='same',activation='selu'))
model.add(Conv2D(128,(5,5),strides=(1,1),padding='same',activation='selu'))
model.add(GlobalAveragePooling2D())
model.add(Dense(256,activation='selu'))
model.add(Dropout(rate=0.1))
model.add(Dense(num_class,activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

#Model Training
saveBestModel = kcallbacks.ModelCheckpoint(filepath=savepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto')
history=model.fit(train_generator,steps_per_epoch=len(train_generator),epochs=epochs,validation_data=validation_generator,
                  validation_steps=len(validation_generator), callbacks=[saveBestModel])

#Plot Training and Testing Accuracies
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('CNN Model Accuracy', fontsize=12, fontweight='bold')
plt.ylabel('Accuracy', fontsize=10, fontweight='bold')
plt.xlabel('Number of Epochs', fontsize=10, fontweight='bold')
plt.legend(['Train', 'Test'], loc='lower right')
plt.grid()
plt.show()

#Plot Training and Testing Loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('CNN Model Loss', fontsize=12, fontweight='bold')
plt.ylabel('Loss', fontsize=10, fontweight='bold')
plt.xlabel('Number of Epochs', fontsize=10, fontweight='bold')
plt.legend(['Train', 'Test'], loc='upper right')
plt.grid()
plt.show()


In [ ]:
#Tuned Hyperparameters for Xception model by Genetic Algorithm
# frozen: Frozen Layers : 128
# f1: Number of units in dense layer: 128
# a1: Activation function : selu
# d1: Drop out : 0.2
# op: Optimizer : adamax
# ep: Number of epochs : 14


input_shape=INPUT_SIZE
num_class=15
epochs=25              #Training is executed for 25 epochs for the better visualization of results
savepath='/content/drive/MyDrive/Transfer_IDS_IIoT/Optimized_Trained_Models/xception.h5'

#Define Xception Model
model_fine_tune = Xception(include_top=False, weights='imagenet', input_shape=input_shape)
for layer in model_fine_tune.layers[:128]:
    layer.trainable = False
for layer in model_fine_tune.layers[128:]:
    layer.trainable = True
model = GlobalAveragePooling2D()(model_fine_tune.output)
model=Dense(units=128,activation='selu')(model)
model=Dropout(0.2)(model)
model = Dense(num_class, activation='softmax')(model)
model = Model(model_fine_tune.input, model, name='xception')
opt = keras.optimizers.Adamax(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

#Model Training
saveBestModel = kcallbacks.ModelCheckpoint(filepath=savepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto')
history=model.fit(train_generator,steps_per_epoch=len(train_generator),epochs=epochs,validation_data=validation_generator,
                  validation_steps=len(validation_generator), callbacks=[saveBestModel])

#Plot Training and Testing Accuracies
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Xception Model Accuracy', fontsize=12, fontweight='bold')
plt.ylabel('Accuracy', fontsize=10, fontweight='bold')
plt.xlabel('Number of Epochs', fontsize=10, fontweight='bold')
plt.legend(['Train', 'Test'], loc='lower right')
plt.grid()
plt.show()

#Plot Training and Testing Loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Xception Model Loss', fontsize=12, fontweight='bold')
plt.ylabel('Loss', fontsize=10, fontweight='bold')
plt.xlabel('Number of Epochs', fontsize=10, fontweight='bold')
plt.legend(['Train', 'Test'], loc='upper right')
plt.grid()
plt.show()


In [ ]:
#Tuned Hyperparameters for VGG16 model by Genetic Algorithm
# frozen: Frozen Layers : 8
# f1: Number of units in dense layer: 128
# a1: Activation function : elu
# d1: Drop out : 0.5
# op: Optimizer : adagrad
# ep: Number of epochs : 11

input_shape=INPUT_SIZE
num_class=15
epochs=25        #Training is executed for 25 epochs for the better visualization of results
savepath='/content/drive/MyDrive/Transfer_IDS_IIoT/Optimized_Trained_Models/VGG16.h5'

#Define VGG16 Model
model_fine_tune = VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
for layer in model_fine_tune.layers[:8]:
    layer.trainable = False
for layer in model_fine_tune.layers[8:]:
    layer.trainable = True
model = GlobalAveragePooling2D()(model_fine_tune.output)
model=Dense(units=128,activation='elu')(model)
model=Dropout(0.5)(model)
model = Dense(num_class, activation='softmax')(model)
model = Model(model_fine_tune.input, model, name='vgg')
opt = keras.optimizers.Adagrad(learning_rate=0.001, initial_accumulator_value=0.1, epsilon=1e-07)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

#Model Training
saveBestModel = kcallbacks.ModelCheckpoint(filepath=savepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto')
history=model.fit(train_generator,steps_per_epoch=len(train_generator),epochs=epochs,validation_data=validation_generator,
                  validation_steps=len(validation_generator), callbacks=[saveBestModel])

#Plot Training and Testing Accuracies
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('VGG16 Model Accuracy', fontsize=12, fontweight='bold')
plt.ylabel('Accuracy', fontsize=10, fontweight='bold')
plt.xlabel('Number of Epochs', fontsize=10, fontweight='bold')
plt.legend(['Train', 'Test'], loc='lower right')
plt.grid()
plt.show()

#Plot Training and Testing Loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('VGG16 Model Loss', fontsize=12, fontweight='bold')
plt.ylabel('Loss', fontsize=10, fontweight='bold')
plt.xlabel('Number of Epochs', fontsize=10, fontweight='bold')
plt.legend(['Train', 'Test'], loc='upper right')
plt.grid()
plt.show()


In [ ]:
#Tuned Hyperparameters for VGG19 model by Genetic Algorithm
# frozen: Frozen Layers : 10
# f1: Number of units in dense layer: 128
# a1: Activation function : relu
# d1: Drop out : 0.3
# op: Optimizer : adamax
# ep: Number of epochs : 16

input_shape=INPUT_SIZE
num_class=15
epochs=25        #Training is executed for 25 epochs for the better visualization of results
savepath='/content/drive/MyDrive/Transfer_IDS_IIoT/Optimized_Trained_Models/VGG19.h5'

#Define VGG19 Model
model_fine_tune = VGG19(include_top=False, weights='imagenet', input_shape=input_shape)
for layer in model_fine_tune.layers[:10]:
    layer.trainable = False
for layer in model_fine_tune.layers[10:]:
    layer.trainable = True
model = GlobalAveragePooling2D()(model_fine_tune.output)
model = Dense(units=128,activation='relu')(model)
model = Dropout(0.3)(model)
model = Dense(num_class, activation='softmax')(model)
model = Model(model_fine_tune.input, model, name='vgg')
opt = keras.optimizers.Adamax(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

#Model Training
saveBestModel = kcallbacks.ModelCheckpoint(filepath=savepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto')
history=model.fit(train_generator,steps_per_epoch=len(train_generator),epochs=epochs,validation_data=validation_generator,
                  validation_steps=len(validation_generator), callbacks=[saveBestModel])

#Plot Training and Testing Accuracies
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('VGG19 Model Accuracy', fontsize=12, fontweight='bold')
plt.ylabel('Accuracy', fontsize=10, fontweight='bold')
plt.xlabel('Number of Epochs', fontsize=10, fontweight='bold')
plt.legend(['Train', 'Test'], loc='lower right')
plt.grid()
plt.show()

#Plot Training and Testing Loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('VGG19 Model Loss', fontsize=12, fontweight='bold')
plt.ylabel('Loss', fontsize=10, fontweight='bold')
plt.xlabel('Number of Epochs', fontsize=10, fontweight='bold')
plt.legend(['Train', 'Test'], loc='upper right')
plt.grid()
plt.show()


In [ ]:
#Tuned Hyperparameters for Inception model by Genetic Algorithm
# frozen: Frozen Layers : 45
# f1: Number of units in dense layer: 128
# a1: Activation function : relu
# d1: Drop out : 0.5
# op: Optimizer : adagrad
# ep: Number of epochs : 19

input_shape=INPUT_SIZE
num_class=15
epochs=25           #Training is executed for 25 epochs for the better visualization of results
savepath='/content/drive/MyDrive/Transfer_IDS_IIoT/Optimized_Trained_Models/inception.h5'

#Define Inception Model
model_fine_tune = InceptionV3(include_top=False, weights='imagenet', input_shape=input_shape)
for layer in model_fine_tune.layers[:45]:
    layer.trainable = False
for layer in model_fine_tune.layers[45:]:
    layer.trainable = True
model = GlobalAveragePooling2D()(model_fine_tune.output)
model=Dense(units=256,activation='relu')(model)
model=Dropout(0.5)(model)
model = Dense(num_class, activation='softmax')(model)
model = Model(model_fine_tune.input, model, name='resnet')
opt = keras.optimizers.Adagrad(learning_rate=0.001, initial_accumulator_value=0.1, epsilon=1e-07)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

#Model Training
saveBestModel = kcallbacks.ModelCheckpoint(filepath=savepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto')
history=model.fit(train_generator,steps_per_epoch=len(train_generator),epochs=epochs,validation_data=validation_generator,
                  validation_steps=len(validation_generator), callbacks=[saveBestModel])

#Plot Training and Testing Accuracies
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Inception Model Accuracy', fontsize=12, fontweight='bold')
plt.ylabel('Accuracy', fontsize=10, fontweight='bold')
plt.xlabel('Number of Epochs', fontsize=10, fontweight='bold')
plt.legend(['Train', 'Test'], loc='lower right')
plt.grid()
plt.show()

#Plot Training and Testing Loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Inception Model Loss', fontsize=12, fontweight='bold')
plt.ylabel('Loss', fontsize=10, fontweight='bold')
plt.xlabel('Number of Epochs', fontsize=10, fontweight='bold')
plt.legend(['Train', 'Test'], loc='upper right')
plt.grid()

In [ ]:
#Tuned Hyperparameters for InceptionResnet model by Genetic Algorithm
# frozen: Frozen Layers : 451
# f1: Number of units in dense layer: 128
# a1: Activation function : selu
# d1: Drop out : 0.3
# op: Optimizer : adamax
# ep: Number of epochs : 12

input_shape=INPUT_SIZE
num_class=15
epochs=25          #Training is executed for 25 epochs for the better visualization of results
savepath='/content/drive/MyDrive/Transfer_IDS_IIoT/Optimized_Trained_Models/inceptionresnetv2.h5'

#Define Inception Resnet V2 Model
model_fine_tune = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=input_shape)
for layer in model_fine_tune.layers[:451]:
    layer.trainable = False
for layer in model_fine_tune.layers[451:]:
    layer.trainable = True
model = GlobalAveragePooling2D()(model_fine_tune.output)
model=Dense(units=128,activation='selu')(model)
model=Dropout(0.3)(model)
model = Dense(num_class, activation='softmax')(model)
model = Model(model_fine_tune.input, model, name='resnet')
opt = keras.optimizers.Adamax(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

#Model Training
saveBestModel = kcallbacks.ModelCheckpoint(filepath=savepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto')
history=model.fit(train_generator,steps_per_epoch=len(train_generator),epochs=epochs,validation_data=validation_generator,
                  validation_steps=len(validation_generator), callbacks=[saveBestModel])

#Plot Training and Testing Accuracies
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('InceptionResnetV2 Model Accuracy', fontsize=12, fontweight='bold')
plt.ylabel('Accuracy', fontsize=10, fontweight='bold')
plt.xlabel('Number of Epochs', fontsize=10, fontweight='bold')
plt.legend(['Train', 'Test'], loc='lower right')
plt.grid()
plt.show()

#Plot Training and Testing Loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('InceptionResnetV2 Model Loss', fontsize=12, fontweight='bold')
plt.ylabel('Loss', fontsize=10, fontweight='bold')
plt.xlabel('Number of Epochs', fontsize=10, fontweight='bold')
plt.legend(['Train', 'Test'], loc='upper right')
plt.grid()


In [ ]:
#Tuned Hyperparameters for EfficientNetB7 model by Genetic Algorithm
# frozen: Frozen Layers : 288
# f1: Number of units in dense layer: 128
# a1: Activation function : selu
# d1: Drop out : 0.4
# op: Optimizer : adam
# ep: Number of epochs : 15

input_shape=INPUT_SIZE
num_class=15
epochs=25      #Training is executed for 25 epochs for the better visualization of results
savepath='/content/drive/MyDrive/Transfer_IDS_IIoT/Optimized_Trained_Models/efficientnetb7.h5'

#Define EfficientNetB7 Model
model_fine_tune = EfficientNetB7(include_top=False, weights='imagenet', input_shape=input_shape)
for layer in model_fine_tune.layers[:288]:
    layer.trainable = False
for layer in model_fine_tune.layers[288:]:
    layer.trainable = True
model = GlobalAveragePooling2D()(model_fine_tune.output)
model=Dense(units=128,activation='selu')(model)
model=Dropout(0.4)(model)
model = Dense(num_class, activation='softmax')(model)
model = Model(model_fine_tune.input, model, name='efficientnetb7')
opt = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

#Model Training
saveBestModel = kcallbacks.ModelCheckpoint(filepath=savepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto')
history=model.fit(train_generator,steps_per_epoch=len(train_generator),epochs=epochs,validation_data=validation_generator,
                  validation_steps=len(validation_generator), callbacks=[saveBestModel])

#Plot Training and Testing Accuracies
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('EfficientNetB7 Model Accuracy', fontsize=12, fontweight='bold')
plt.ylabel('Accuracy', fontsize=10, fontweight='bold')
plt.xlabel('Number of Epochs', fontsize=10, fontweight='bold')
plt.legend(['Train', 'Test'], loc='lower right')
plt.grid()
plt.show()

#Plot Training and Testing Loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('EfficientNetB7 Model Loss', fontsize=12, fontweight='bold')
plt.ylabel('Loss', fontsize=10, fontweight='bold')
plt.xlabel('Number of Epochs', fontsize=10, fontweight='bold')
plt.legend(['Train', 'Test'], loc='upper right')
plt.grid()


In [ ]:
#Tuned Hyperparameters for  EfficientNetV2L model by Genetic Algorithm
# frozen: Frozen Layers : 316
# f1: Number of units in dense layer: 128
# a1: Activation function : selu
# d1: Drop out : 0.5
# op: Optimizer : adam
# ep: Number of epochs : 19

input_shape=INPUT_SIZE
num_class=15
epochs=25          #Training is executed for 25 epochs for the better visualization of results
savepath='/content/drive/MyDrive/Transfer_IDS_IIoT/Optimized_Trained_Models/efficientnetv2l.h5'

#Define EfficientNetB7 Model
model_fine_tune = EfficientNetV2L(include_top=False, weights='imagenet', input_shape=input_shape)
for layer in model_fine_tune.layers[:316]:
    layer.trainable = False
for layer in model_fine_tune.layers[316:]:
    layer.trainable = True
model = GlobalAveragePooling2D()(model_fine_tune.output)
model=Dense(units=128,activation='selu')(model)
model=Dropout(0.5)(model)
model = Dense(num_class, activation='softmax')(model)
model = Model(model_fine_tune.input, model, name='efficientnetv2-l')
opt = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

#Model Training
saveBestModel = kcallbacks.ModelCheckpoint(filepath=savepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto')
history=model.fit(train_generator,steps_per_epoch=len(train_generator),epochs=epochs,validation_data=validation_generator,
                  validation_steps=len(validation_generator), callbacks=[saveBestModel])

#Plot Training and Testing Accuracies
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('EfficientNetV2L Model Accuracy', fontsize=12, fontweight='bold')
plt.ylabel('Accuracy', fontsize=10, fontweight='bold')
plt.xlabel('Number of Epochs', fontsize=10, fontweight='bold')
plt.legend(['Train', 'Test'], loc='lower right')
plt.grid()
plt.show()

#Plot Training and Testing Loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('EfficientNetV2L Model Loss', fontsize=12, fontweight='bold')
plt.ylabel('Loss', fontsize=10, fontweight='bold')
plt.xlabel('Number of Epochs', fontsize=10, fontweight='bold')
plt.legend(['Train', 'Test'], loc='upper right')
plt.grid()

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import keras
from keras.models import Model,load_model
from keras import Input
from keras.layers import concatenate,Dense,Flatten,Dropout
from keras.preprocessing.image import  ImageDataGenerator
import keras.callbacks as kcallbacks
import os
import math
from keras.utils import plot_model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, LearningRateScheduler
from keras.optimizers import SGD
import operator
import numpy as np
from PIL import Image
from collections import defaultdict

In [ ]:
#Generate Images from Test Set
TARGET_SIZE=(224,224)
INPUT_SIZE=(224,224,3)
BATCHSIZE=32

test_datagen = ImageDataGenerator(rescale=1./255)


validation_generator = test_datagen.flow_from_directory(
        '/content/drive/MyDrive/Transfer_IDS_IIoT/Datasets/test_A',
        target_size=TARGET_SIZE,
        batch_size=BATCHSIZE,
        class_mode='categorical')

In [ ]:
#generate labels indicating attack (1) or normal (0)
label=validation_generator.class_indices
label={v: k for k, v in label.items()}

In [2]:
print(label)

NameError: name 'label' is not defined

In [ ]:
#Read Images from Test Folders
rootdir = '/content/drive/MyDrive/Transfer_IDS_IIoT/Datasets/test_A'
true = []          #True Labels
test_images=[]
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if not (file.endswith(".jpeg"))|(file.endswith(".jpg"))|(file.endswith(".png")):
            continue
        true.append(subdir.split('/')[-1])
        test_images.append(os.path.join(subdir, file))

print(true[0],test_images[0])

In [ ]:
#load model 1: Generic CNN Model
cnn_model=load_model('/content/drive/MyDrive/Transfer_IDS_IIoT/Optimized_Trained_Models/generic_cnn.h5')

In [ ]:
#load model 2: Xception
xception_model=load_model('/content/drive/MyDrive/Transfer_IDS_IIoT/Optimized_Trained_Models/xception.h5')

In [ ]:
#load model 3: VGG16
vgg_model=load_model('/content/drive/MyDrive/Transfer_IDS_IIoT/Optimized_Trained_Models/VGG16.h5')

In [ ]:
#load model 4: VGG19
vgg19_model=load_model('/content/drive/MyDrive/Transfer_IDS_IIoT/Optimized_Trained_Models/VGG19.h5')

In [ ]:
#load model 5: inception
incep_model=load_model('/content/drive/MyDrive/Transfer_IDS_IIoT/Optimized_Trained_Models/inception.h5')

In [ ]:
#load model 6: InceptionResnetV2
incepresnetv2_model=load_model('/content/drive/MyDrive/Transfer_IDS_IIoT/Optimized_Trained_Models/inceptionresnetv2.h5')

In [ ]:
#load model 7: EfficientB7
efficientb7_model=load_model('/content/drive/MyDrive/Transfer_IDS_IIoT/Optimized_Trained_Models/efficientnetb7.h5')

In [ ]:
#load model 8: EfficientNetV2L
efficientnetv2l_model=load_model('/content/drive/MyDrive/Transfer_IDS_IIoT/Optimized_Trained_Models/efficientnetv2l.h5')

In [3]:
pip install scikit-plot

In [ ]:
import seaborn as sn
import pandas as pd
import scikitplot as skplt
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Results generation by Generic CNN


%%time
import time
predict=[]
length=len(test_images)
t1 = time.time()
for i in range(length):
    input_img=test_images[i]
    test_batch=[]
    this_img=np.array(Image.open(input_img))/255 #Read all the images from test set

    test_shape=(1,)+this_img.shape
    this_img=this_img.reshape(test_shape)
    cnn_model_prediction=cnn_model.predict(this_img) #Input image processing using trained master model
    #Results generation by Generic CNN
    prob=cnn_model_prediction[0,np.argmax(cnn_model_prediction,axis=1)[0]]
    res=label[np.argmax(cnn_model_prediction,axis=1)[0]]
    predict.append(res)

In [ ]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,cohen_kappa_score
acc=accuracy_score(true,predict)
pre=precision_score(true,predict,average='weighted')
re=recall_score(true,predict,average='weighted')
f1=f1_score(true,predict,average='weighted')
cks=cohen_kappa_score(true,predict)
print('CNN Accuracy: %s'%acc)
print('Precision: %s'%pre)
print('Recall: %s'%re)
print('F1 Score: %s'%f1)
print('Cohen Kappa Score: %s'%cks)

from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
target_names = ['Backdoor','DDoS_HTTP','DDoS_ICMP','DDoS_TCP','DDoS_UDP','Fingerprinting','MITM', 'Normal','Password','Port_Scanning','Ransomware','SQL_injection','Uploading','Vulnerability_scanner','XSS']
skplt.metrics.plot_confusion_matrix(true, predict, normalize=True, x_tick_rotation=90,figsize=(10,10),cmap='Blues' )
plt.show()
print(classification_report(true, predict, target_names=target_names,digits=4))

In [ ]:
 #Results generation by Xception


 %%time
import time
predict=[]
length=len(test_images)
t1 = time.time()
for i in range(length):
    input_img=test_images[i]
    test_batch=[]
    this_img=np.array(Image.open(input_img))/255 #Read all the images from test set

    test_shape=(1,)+this_img.shape
    this_img=this_img.reshape(test_shape)
    xception_model_prediction=xception_model.predict(this_img) #Input image processing using trained master model
    #Results generation by Xception
    prob=xception_model_prediction[0,np.argmax(xception_model_prediction,axis=1)[0]]
    res=label[np.argmax(xception_model_prediction,axis=1)[0]]
    predict.append(res)

In [ ]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,cohen_kappa_score
acc=accuracy_score(true,predict)
pre=precision_score(true,predict,average='weighted')
re=recall_score(true,predict,average='weighted')
f1=f1_score(true,predict,average='weighted')
cks=cohen_kappa_score(true,predict)
print('Xception Accuracy: %s'%acc)
print('Precision: %s'%pre)
print('Recall: %s'%re)
print('F1 Score: %s'%f1)
print('Cohen Kappa Score: %s'%cks)

from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
target_names = ['Backdoor','DDoS_HTTP','DDoS_ICMP','DDoS_TCP','DDoS_UDP','Fingerprinting','MITM', 'Normal','Password','Port_Scanning','Ransomware','SQL_injection','Uploading','Vulnerability_scanner','XSS']
skplt.metrics.plot_confusion_matrix(true, predict, normalize=True, x_tick_rotation=90,figsize=(10,10),cmap='Blues' )
plt.show()
print(classification_report(true, predict, target_names=target_names,digits=4))

In [ ]:
 #Results generation by VGG16


%%time
predict=[]
length=len(test_images)
t1 = time.time()
for i in range(length):
    input_img=test_images[i]
    test_batch=[]
    this_img=np.array(Image.open(input_img))/255 #Read all the images from test set

    test_shape=(1,)+this_img.shape
    this_img=this_img.reshape(test_shape)
    vgg_model_prediction=vgg_model.predict(this_img) #Input image processing using trained master model
    #Results generation by VGG16
    prob=vgg_model_prediction[0,np.argmax(vgg_model_prediction,axis=1)[0]]
    res=label[np.argmax(vgg_model_prediction,axis=1)[0]]
    predict.append(res)

In [ ]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,cohen_kappa_score
acc=accuracy_score(true,predict)
pre=precision_score(true,predict,average='weighted')
re=recall_score(true,predict,average='weighted')
f1=f1_score(true,predict,average='weighted')
cks=cohen_kappa_score(true,predict)
print('VGG16 Accuracy: %s'%acc)
print('Precision: %s'%pre)
print('Recall: %s'%re)
print('F1 Score: %s'%f1)
print('Cohen Kappa Score: %s'%cks)

from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
target_names = ['Backdoor','DDoS_HTTP','DDoS_ICMP','DDoS_TCP','DDoS_UDP','Fingerprinting','MITM', 'Normal','Password','Port_Scanning','Ransomware','SQL_injection','Uploading','Vulnerability_scanner','XSS']
skplt.metrics.plot_confusion_matrix(true, predict, normalize=True, x_tick_rotation=90,figsize=(10,10),cmap='Blues' )
plt.show()
print(classification_report(true, predict, target_names=target_names,digits=4))

In [ ]:
#Results generation by Generic VGG19


%%time
predict=[]
length=len(test_images)
t1 = time.time()
for i in range(length):
    input_img=test_images[i]
    test_batch=[]
    this_img=np.array(Image.open(input_img))/255 #Read all the images from test set

    test_shape=(1,)+this_img.shape
    this_img=this_img.reshape(test_shape)
    vgg19_model_prediction=vgg19_model.predict(this_img) #Input image processing using trained master model
    #Results generation by Generic VGG19
    prob=vgg19_model_prediction[0,np.argmax(vgg19_model_prediction,axis=1)[0]]
    res=label[np.argmax(vgg19_model_prediction,axis=1)[0]]
    predict.append(res)

In [ ]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,cohen_kappa_score
acc=accuracy_score(true,predict)
pre=precision_score(true,predict,average='weighted')
re=recall_score(true,predict,average='weighted')
f1=f1_score(true,predict,average='weighted')
cks=cohen_kappa_score(true,predict)
print('VGG19 Accuracy: %s'%acc)
print('Precision: %s'%pre)
print('Recall: %s'%re)
print('F1 Score: %s'%f1)
print('Cohen Kappa Score: %s'%cks)

from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
target_names = ['Backdoor','DDoS_HTTP','DDoS_ICMP','DDoS_TCP','DDoS_UDP','Fingerprinting','MITM', 'Normal','Password','Port_Scanning','Ransomware','SQL_injection','Uploading','Vulnerability_scanner','XSS']
skplt.metrics.plot_confusion_matrix(true, predict, normalize=True, x_tick_rotation=90,figsize=(10,10),cmap='Blues' )
plt.show()
print(classification_report(true, predict, target_names=target_names,digits=4))

In [ ]:
#Results generation by Inception



%%time
predict=[]
length=len(test_images)
t1 = time.time()
for i in range(length):
    input_img=test_images[i]
    test_batch=[]
    this_img=np.array(Image.open(input_img))/255 #Read all the images from test set

    test_shape=(1,)+this_img.shape
    this_img=this_img.reshape(test_shape)
    incep_model_prediction=incep_model.predict(this_img) #Input image processing using trained master model
    #Results generation by Inception
    prob=incep_model_prediction[0,np.argmax(incep_model_prediction,axis=1)[0]]
    res=label[np.argmax(incep_model_prediction,axis=1)[0]]
    predict.append(res)

In [ ]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,cohen_kappa_score
acc=accuracy_score(true,predict)
pre=precision_score(true,predict,average='weighted')
re=recall_score(true,predict,average='weighted')
f1=f1_score(true,predict,average='weighted')
cks=cohen_kappa_score(true,predict)
print('Inception Accuracy: %s'%acc)
print('Precision: %s'%pre)
print('Recall: %s'%re)
print('F1 Score: %s'%f1)
print('Cohen Kappa Score: %s'%cks)

from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
target_names = ['Backdoor','DDoS_HTTP','DDoS_ICMP','DDoS_TCP','DDoS_UDP','Fingerprinting','MITM', 'Normal','Password','Port_Scanning','Ransomware','SQL_injection','Uploading','Vulnerability_scanner','XSS']
skplt.metrics.plot_confusion_matrix(true, predict, normalize=True, x_tick_rotation=90,figsize=(10,10),cmap='Blues' )
plt.show()
print(classification_report(true, predict, target_names=target_names,digits=4))

In [ ]:
#Results generation by InceptionResnetV2


%%time
predict=[]
length=len(test_images)
t1 = time.time()
for i in range(length):
    input_img=test_images[i]
    test_batch=[]
    this_img=np.array(Image.open(input_img))/255 #Read all the images from test set

    test_shape=(1,)+this_img.shape
    this_img=this_img.reshape(test_shape)
    incepresnetv2_model_prediction=incepresnetv2_model.predict(this_img) #Input image processing using trained master model
    #Results generation by InceptionResnetV2
    prob=incepresnetv2_model_prediction[0,np.argmax(incepresnetv2_model_prediction,axis=1)[0]]
    res=label[np.argmax(incepresnetv2_model_prediction,axis=1)[0]]
    predict.append(res)

In [ ]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,cohen_kappa_score
acc=accuracy_score(true,predict)
pre=precision_score(true,predict,average='weighted')
re=recall_score(true,predict,average='weighted')
f1=f1_score(true,predict,average='weighted')
cks=cohen_kappa_score(true,predict)
print('InseptionResnetV2 Accuracy: %s'%acc)
print('Precision: %s'%pre)
print('Recall: %s'%re)
print('F1 Score: %s'%f1)
print('Cohen Kappa Score: %s'%cks)

from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
target_names = ['Backdoor','DDoS_HTTP','DDoS_ICMP','DDoS_TCP','DDoS_UDP','Fingerprinting','MITM', 'Normal','Password','Port_Scanning','Ransomware','SQL_injection','Uploading','Vulnerability_scanner','XSS']
skplt.metrics.plot_confusion_matrix(true, predict, normalize=True, x_tick_rotation=90,figsize=(10,10),cmap='Blues' )
plt.show()
print(classification_report(true, predict, target_names=target_names,digits=4))

In [ ]:
#Results generation by EfficientNetB7


%%time
predict=[]
length=len(test_images)
t1 = time.time()
for i in range(length):
    input_img=test_images[i]
    test_batch=[]
    this_img=np.array(Image.open(input_img))/255 #Read all the images from test set

    test_shape=(1,)+this_img.shape
    this_img=this_img.reshape(test_shape)
    efficientb7_model_prediction=efficientb7_model.predict(this_img) #Input image processing using trained master model
    #Results generation by EfficientNetB7
    prob=efficientb7_model_prediction[0,np.argmax(efficientb7_model_prediction,axis=1)[0]]
    res=label[np.argmax(efficientb7_model_prediction,axis=1)[0]]
    predict.append(res)

In [ ]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,cohen_kappa_score
acc=accuracy_score(true,predict)
pre=precision_score(true,predict,average='weighted')
re=recall_score(true,predict,average='weighted')
f1=f1_score(true,predict,average='weighted')
cks=cohen_kappa_score(true,predict)
print('Efficient Net B7 Accuracy: %s'%acc)
print('Precision: %s'%pre)
print('Recall: %s'%re)
print('F1 Score: %s'%f1)
print('Cohen Kappa Score: %s'%cks)

from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
target_names = ['Backdoor','DDoS_HTTP','DDoS_ICMP','DDoS_TCP','DDoS_UDP','Fingerprinting','MITM', 'Normal','Password','Port_Scanning','Ransomware','SQL_injection','Uploading','Vulnerability_scanner','XSS']
skplt.metrics.plot_confusion_matrix(true, predict, normalize=True, x_tick_rotation=90,figsize=(10,10),cmap='Blues' )
plt.show()
print(classification_report(true, predict, target_names=target_names,digits=4))

In [ ]:
#Results generation by EfficientNetV2L


%%time
predict=[]
length=len(test_images)
t1 = time.time()
for i in range(length):
    input_img=test_images[i]
    test_batch=[]
    this_img=np.array(Image.open(input_img))/255 #Read all the images from test set

    test_shape=(1,)+this_img.shape
    this_img=this_img.reshape(test_shape)
    efficientnetv2l_model_prediction=efficientnetv2l_model.predict(this_img) #Input image processing using trained master model
    #Results generation by EfficientNetV2L
    prob=efficientnetv2l_model_prediction[0,np.argmax(efficientnetv2l_model_prediction,axis=1)[0]]
    res=label[np.argmax(efficientnetv2l_model_prediction,axis=1)[0]]
    predict.append(res)

In [ ]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,cohen_kappa_score
acc=accuracy_score(true,predict)
pre=precision_score(true,predict,average='weighted')
re=recall_score(true,predict,average='weighted')
f1=f1_score(true,predict,average='weighted')
cks=cohen_kappa_score(true,predict)
print('Efficient NetV2L Accuracy: %s'%acc)
print('Precision: %s'%pre)
print('Recall: %s'%re)
print('F1 Score: %s'%f1)
print('Cohen Kappa Score: %s'%cks)

from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
target_names = ['Backdoor','DDoS_HTTP','DDoS_ICMP','DDoS_TCP','DDoS_UDP','Fingerprinting','MITM', 'Normal','Password','Port_Scanning','Ransomware','SQL_injection','Uploading','Vulnerability_scanner','XSS']
skplt.metrics.plot_confusion_matrix(true, predict, normalize=True, x_tick_rotation=90,figsize=(10,10),cmap='Blues' )
plt.show()
print(classification_report(true, predict, target_names=target_names,digits=4))

In [ ]:
##################################################


import time
predict=[]
length=len(test_images)
t1 = time.time()
for i in range((length//127)+1):
    input_img=test_images[127*i:127*(i+1)]
    test_batch=[]
    for path in input_img:
        this_img=np.array(Image.open(path))/255
        test_batch.append(this_img)
    #Results generation by Generic CNN
    cnn_model_prediction=cnn_model.predict(np.array(test_batch))
    cnn_model_prediction=list(np.argmax(cnn_model_prediction,axis=1))
    cnn_model_prediction=[label[con] for con in cnn_model_prediction]
    #Results generation by VGG19
    vgg19_model_prediction=vgg19_model.predict(np.array(test_batch))
    vgg19_model_prediction=list(np.argmax(vgg19_model_prediction,axis=1))
    vgg19_model_prediction=[label[con] for con in vgg19_model_prediction]
    #Results generation by Inception
    incep_model_prediction=incep_model.predict(np.array(test_batch))
    incep_model_prediction=list(np.argmax(incep_model_prediction,axis=1))
    incep_model_prediction=[label[con] for con in incep_model_prediction]
    #Results generation by InceptionResnetV2
    incepresnetv2_model_prediction=incepresnetv2_model.predict(np.array(test_batch))
    incepresnetv2_model_prediction=list(np.argmax(incepresnetv2_model_prediction,axis=1))
    incepresnetv2_model_prediction=[label[con] for con in incepresnetv2_model_prediction]
    #Results generation by EfficientNetV2L
    efficientnetv2l_model_prediction=efficientnetv2l_model.predict(np.array(test_batch))
    efficientnetv2l_model_prediction=list(np.argmax(efficientnetv2l_model_prediction,axis=1))
    efficientnetv2l_model_prediction=[label[con] for con in efficientnetv2l_model_prediction]

    #Results integration of Five best-performing models
    predict_batch=[]
    for i,j,k,p,q in zip(cnn_model_prediction, vgg19_model_prediction,incep_model_prediction,incepresnetv2_model_prediction,efficientnetv2l_model_prediction,):
        count=defaultdict(int)
        count[i]+=1
        count[j]+=1
        count[k]+=1
        count[p]+=1
        count[q]+=1
        #Arrang the predicted results in descending order
        predict_one=sorted(count.items(), key=operator.itemgetter(1),reverse=True)[0][0]
        predict_batch.append(predict_one)
    predict.append(predict_batch)
t2 = time.time()
print('The Testing Time is :%f seconds' % (t2-t1))

In [ ]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,cohen_kappa_score
acc=accuracy_score(true,predict)
pre=precision_score(true,predict,average='weighted')
re=recall_score(true,predict,average='weighted')
f1=f1_score(true,predict,average='weighted')
cks=cohen_kappa_score(true,predict)
print('Ensemble Accuracy: %s'%acc)
print('Precision: %s'%pre)
print('Recall: %s'%re)
print('F1 Score: %s'%f1)
print('Cohen Kappa Score: %s'%cks)

from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
target_names = ['Backdoor','DDoS_HTTP','DDoS_ICMP','DDoS_TCP','DDoS_UDP','Fingerprinting','MITM', 'Normal','Password','Port_Scanning','Ransomware','SQL_injection','Uploading','Vulnerability_scanner','XSS']
skplt.metrics.plot_confusion_matrix(true, predict, normalize=True, x_tick_rotation=90,figsize=(10,10),cmap='Blues' )
plt.show()
print(classification_report(true, predict, target_names=target_names,digits=4))